In [12]:
import pandas as pd
pd.set_option('display.max_rows',500)

In [3]:
df=pd.read_parquet("s3://unitrento/dc_events_timestamp/year=2021/month=12/day=21/output.snappy.parquet")

In [27]:
class ColumnsInput:
    barcode = 'ddc_barcode'
    ipcode = 'ddc_ipcode'
    machine = 'ddc_mch_code'
    machine_side = 'ddc_mch_side'
    event = 'ddc_ev_subcode'
    time = 'ddc_ev_timestamp'
    
    
class ColumnsOutput:
    c_machine = 'c_machine'    

In [28]:
df[ColumnsOutput.c_machine] = df[ColumnsInput.machine].str.startswith('C')
df.columns

Index(['ddc_barcode', 'ddc_ipcode', 'ddc_mch_code', 'ddc_mch_side',
       'ddc_ev_subcode', 'ddc_ev_timestamp', 'c_machine'],
      dtype='object')

In [6]:
df[[ColumnsInput.barcode]].describe()

,ddc_barcode
count,73942
unique,2214
top,9435414666
freq,39


### How many tyre per press?
Every machine has two sides that can work in parallel, the entire cycle is usually between 20-40 minutes.

In [13]:
df.groupby([ColumnsInput.machine,ColumnsInput.machine_side])[ColumnsInput.barcode].nunique()

ddc_mch_code  ddc_mch_side
1201          L               25
              R               25
1202          L               29
              R               26
1204          L                2
1206          L               16
              R               14
1207          L               16
              R               16
1302          L               35
              R               32
1303          L               21
1304          R               20
1307          L               30
              R               30
1401          L                5
              R               14
1405          L               25
              R               24
1503          L               29
1504          R               21
1505          L               29
              R               24
1506          R               18
1507          L                1
C103          L               19
              R               19
C104          L               17
              R               17
C105          L 

### How many kind of tyres per machine?
Usually a single press cures a single recipe (ipcode) during a single day.
Changes of ipcode are avoided, because they requires a manual operation, thus human time

In [9]:
df.groupby([ColumnsInput.machine])[ColumnsInput.ipcode].nunique()

ddc_mch_code
1201    1
1202    1
1204    1
1206    1
1207    1
1302    2
1303    1
1304    1
1307    1
1401    1
1405    1
1503    1
1504    1
1505    2
1506    1
1507    1
C103    1
C104    1
C105    1
C109    1
C110    1
C111    1
C201    1
C202    1
C203    1
C204    1
C207    1
C208    1
C209    1
C210    1
C301    1
C302    1
C303    1
C304    1
C305    1
C310    1
C311    1
C402    1
C403    1
C404    1
C405    1
C407    1
C410    1
C411    1
C412    1
C502    1
C503    1
C508    1
C509    1
C511    1
C512    1
C601    1
C602    1
C603    1
C606    1
C610    1
C612    1
Name: ddc_ipcode, dtype: int64

### How to identify the 'old' presses?

In [17]:
#New one should have higher production
df_by_machine = df.groupby([ColumnsInput.machine])[ColumnsInput.barcode].nunique().reset_index()
col_c_machine = 'c_machine'
df_by_machine[col_c_machine] = df_by_machine[ColumnsInput.machine].str.startswith('C')
df_by_machine.groupby(col_c_machine)[ColumnsInput.barcode].mean()

c_machine
False    32.937500
True     41.146341
Name: ddc_barcode, dtype: float64

In [30]:
#Curing time between sides will be different in new machines
df.groupby([ColumnsOutput.c_machine])[ColumnsInput.event].nunique()

c_machine
False    39
True     45
Name: ddc_ev_subcode, dtype: int64

In [42]:
df.dt['min']

AttributeError: 'DataFrame' object has no attribute 'dt'

In [50]:
df_min_max_time = df.groupby([ColumnsInput.machine,ColumnsOutput.c_machine,ColumnsInput.barcode])[ColumnsInput.time].agg(['min','max']).reset_index()
col = 'delta_time'
df_min_max_time[col] = (df_min_max_time['max'] - df_min_max_time['min']).dt.total_seconds()
df_min_max_time.groupby(ColumnsOutput.c_machine)[col].mean()
#df_min_max_time.groupby(ColumnsOutput.c_machine).mean()

c_machine
False    1112.444023
True     1124.307647
Name: delta_time, dtype: float64

In [37]:
df.sort_values(ColumnsInput.time).groupby(ColumnsInput.barcode)[ColumnsInput.event].agg(['first','last']).value_counts()

first                   last                 
LO_LOADER_IN_PRESS      UN_FORK_OUT              1348
                        UN_UNLOADER_OUT           703
                        UN_TCR_DOWN                56
CURING_OFF              UN_FORK_OUT                33
                        UN_UNLOADER_OUT            24
LO_LOADER_IN_PRESS      UN_TCR_UP_LMR_DOWN          6
OP_PRESS_MOVEMENT_UP_3  UN_FORK_OUT                 6
OP_PRESS_STOP_PAUSE_3   UN_FORK_OUT                 5
CURING_OFF              UN_TCR_DOWN                 5
LO_LOADER_IN_PRESS      UN_UNLOADER_UP              4
                        UN_VACUUM                   4
                        OP_PRESS_LOCKED             3
                        OP_PRESS_STOP_PAUSE_3       3
CL_PRESS_DOWN           UN_FORK_OUT                 2
OP_SQUEEZE_OFF          UN_UNLOADER_OUT             2
CURING_OFF              CURING_ON                   2
LO_LOADER_IN_PRESS      OP_UNLOCK_PRESS             2
OP_PRESS_MOVEMENT_UP_1  UN_FORK_OUT 